<a href="https://colab.research.google.com/github/mkkim007/Hackathon-KaggleFestival/blob/main/mkkim/Restaurant_Visitor_Forecasting_EDA(weather).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!mkdir /content/data
!mkdir /content/Weather

In [5]:
!cp /content/gdrive/MyDrive/Data/recruit-restaurant-visitor-forecasting.zip /content/data

In [6]:
!unzip -qq /content/data/recruit-restaurant-visitor-forecasting.zip -d /content/data

In [7]:
!cp /content/gdrive/MyDrive/Data/recruit-restaurant-visitor-forecasting-weather.zip /content/Weather
!unzip -qq /content/Weather/recruit-restaurant-visitor-forecasting-weather.zip -d /content/Weather

In [8]:
import warnings
warnings.filterwarnings("ignore")

import time
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime
from datetime import date, timedelta
from sklearn.preprocessing import LabelEncoder

import missingno as msno

import folium
from folium import plugins as folium_plugins
from folium import features


from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb
from scipy.stats import norm #Analysis 
from scipy import stats #Analysis 
import scipy as sp
from sklearn.preprocessing import StandardScaler #Analysis 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import gc

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score , cross_validate

# 날씨 데이터

In [9]:
pd.set_option('display.expand_frame_repr', False)
warnings.simplefilter(action='ignore', category=FutureWarning)

weather_set_dir = '/content/Weather/'
weather_data_dir = '{}/1-1-16_5-31-17_Weather/1-1-16_5-31-17_Weather/'.format(weather_set_dir)
original_data_dir = '/content/data'

weather_columns = [
    'avg_temperature', 'high_temperature', 'low_temperature', 'precipitation',
    'hours_sunlight', 'solar_radiation', 'deepest_snowfall', 'total_snowfall', 'avg_wind_speed',
    'avg_vapor_pressure', 'avg_local_pressure', 'avg_humidity', 'avg_sea_pressure', 
    'cloud_cover'
]

In [10]:
air_store_info = pd.read_csv('{}/air_store_info_with_nearest_active_station.csv'.format(weather_set_dir))
hpg_store_info = pd.read_csv('{}/hpg_store_info_with_nearest_active_station.csv'.format(weather_set_dir))

air_station_distances = pd.read_csv('{}/air_station_distances.csv'.format(weather_set_dir))
hpg_station_distances = pd.read_csv('{}/hpg_station_distances.csv'.format(weather_set_dir))

weather_stations = pd.read_csv('{}/weather_stations.csv'.format(weather_set_dir))
nearby_active_stations = pd.read_csv('{}/nearby_active_stations.csv'.format(weather_set_dir))
feature_manifest = pd.read_csv('{}/feature_manifest.csv'.format(weather_set_dir))

기존에 있는 info와 날씨 데이터에 있는 info가 같은지 확인!

In [11]:
original_as_info = pd.read_csv('{}/air_store_info.csv.zip'.format(original_data_dir))
original_hs_info = pd.read_csv('{}/hpg_store_info.csv.zip'.format(original_data_dir))

display(original_as_info.head(5))
display(original_hs_info.head(5))

print('Air Equal: {}'.format(original_as_info.equals(air_store_info[original_as_info.columns])))
print('HPG Equal: {}'.format(original_hs_info.equals(hpg_store_info[original_hs_info.columns])))

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


Air Equal: True
HPG Equal: True


In [14]:
air_nearest = pd.read_csv(                                                                                                                
        '{}air_store_info_with_nearest_active_station.csv'.format(weather_set_dir))                                                              
unique_air_store_ids = list(data.store_id.unique())
weather_keep_columns = ['precipitation', 'avg_temperature']
dataset_with_weather = data.copy()
for column in weather_keep_columns:                                                                                                       
    dataset_with_weather[column] = np.nan   

In [15]:
dataset_with_weather

,store_id,visit_date,visitors,id,dow,air_genre_name,air_area_name,latitude,longitude,air_area_name0,holiday_flg,holiday_flg2,precipitation,avg_temperature
0,air_ba937bf13d40fb24,2016-01-13,3.258097,air_ba937bf13d40fb24_2016-01-13,2,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-14,3.496508,air_ba937bf13d40fb24_2016-01-14,3,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-15,3.401197,air_ba937bf13d40fb24_2016-01-15,4,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0,NaN,NaN
3,air_ba937bf13d40fb24,2016-01-16,3.135494,air_ba937bf13d40fb24_2016-01-16,5,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,1,NaN,NaN
4,air_ba937bf13d40fb24,2016-01-18,1.945910,air_ba937bf13d40fb24_2016-01-18,0,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284122,air_fff68b929994bfbd,2017-05-27,0.000000,air_fff68b929994bfbd_2017-05-27,5,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,1,NaN,NaN
284123,air_fff68b929994bfbd,2017-05-28,0.000000,air_fff68b929994bfbd_2017-05-28,6,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,1,NaN,NaN
284124,air_fff68b929994bfbd,2017-05-29,0.000000,air_fff68b929994bfbd_2017-05-29,0,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,0,NaN,NaN
284125,air_fff68b929994bfbd,2017-05-30,0.000000,air_fff68b929994bfbd_2017-05-30,1,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,0,NaN,NaN


In [17]:
print(len(air_nearest['air_store_id'].unique()))
print(len(data['store_id'].unique()))

829
829


In [20]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   visit_date          517 non-null    datetime64[ns]
 1   avg_temperature     517 non-null    float64       
 2   high_temperature    517 non-null    float64       
 3   low_temperature     517 non-null    float64       
 4   precipitation       268 non-null    float64       
 5   hours_sunlight      517 non-null    float64       
 6   solar_radiation     517 non-null    float64       
 7   deepest_snowfall    4 non-null      float64       
 8   total_snowfall      1 non-null      float64       
 9   avg_wind_speed      517 non-null    float64       
 10  avg_vapor_pressure  516 non-null    float64       
 11  avg_local_pressure  517 non-null    float64       
 12  avg_humidity        516 non-null    float64       
 13  avg_sea_pressure    517 non-null    float64       

In [ ]:
weather_data_all
for air_id in unique_air_store_ids:                                                                                                       
    station = air_nearest[air_nearest.air_store_id == air_id].station_id.iloc[0]

    weather_data = pd.read_csv(weather_data_dir + station + '.csv', parse_dates=['calendar_date']).rename(columns={'calendar_date': 'visit_date'})                                                                                                                                                       
    this_store = data.store_id == air_id 
   
    print(weather_data.info())                                                                                          
    merged = data[this_store].merge(weather_data, on='visit_date', how='left')                                                         
                                                                                                                                              
    for column in weather_keep_columns:                                                                                                   
        dataset_with_weather.loc[this_store, column] = merged[column]
        dataset_with_weather = dataset_with_weather[]

In [22]:
for air_id in unique_air_store_ids:                                                                                                       
    station = air_nearest[air_nearest.air_store_id == air_id].station_id.iloc[0]

    weather_data = pd.read_csv(weather_data_dir + station + '.csv', parse_dates=['calendar_date']).rename(columns={'calendar_date': 'visit_date'})                                                                                                                                           
    data['visit_date'] = pd.to_datetime(data['visit_date']) 
    d_id = data['visit_date','store_id']                                                                                                                                         
    this_store = data.store_id == air_id 
   
    print(weather_data.info())                                                                                          
    merged = data[this_store].merge(weather_data, on='visit_date', how='left')                                                         
                                                                                                                                              
    for column in weather_keep_columns:                                                                                                   
        dataset_with_weather.loc[this_store, column] = merged[column]
        dataset_with_weather = dataset_with_weather[]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 9   avg_wind_speed      517 non-null    float64       
 10  avg_vapor_pressure  0 non-null      float64       
 11  avg_local_pressure  0 non-null      float64       
 12  avg_humidity        0 non-null      float64       
 13  avg_sea_pressure    0 non-null      float64       
 14  cloud_cover         0 non-null      float64       
dtypes: datetime64[ns](1), float64(14)
memory usage: 60.7 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   visit_date          517 non-null    datetime64[ns]
 1   avg_temperature     517 non-null    float64       
 2   high_temperature    517 non-null    float64       
 3   low_temperature     517 non-null    float64       
 4   precipitation       517 non-null    float64       
 5   hours_sunlight      0 non-null      float64       
 6   sola

In [ ]:
dataset_with_weather['precipitation']=dataset_with_weather['precipitation'].fillna(0)

In [19]:
dataset_with_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284127 entries, 0 to 284126
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   store_id         284127 non-null  object 
 1   visit_date       284127 non-null  object 
 2   visitors         284127 non-null  float64
 3   id               284127 non-null  object 
 4   dow              284127 non-null  int64  
 5   air_genre_name   284127 non-null  int64  
 6   air_area_name    284127 non-null  object 
 7   latitude         284127 non-null  float64
 8   longitude        284127 non-null  float64
 9   air_area_name0   284127 non-null  int64  
 10  holiday_flg      284127 non-null  int64  
 11  holiday_flg2     284127 non-null  int64  
 12  precipitation    204 non-null     float64
 13  avg_temperature  391 non-null     float64
dtypes: float64(5), int64(5), object(4)
memory usage: 32.5+ MB


In [ ]:
def add_weather(dataset):                                                                                                                     
    print('Adding weather...')                                                                                                                
    air_nearest = pd.read_csv(                                                                                                                
        '{}air_store_info_with_nearest_active_station.csv'.format(weather_set_dir))                                                              
    unique_air_store_ids = list(dataset.store_id.unique())                                                                                
                                                                                                                                                                                                            
    weather_keep_columns = ['precipitation', 'avg_temperature']                                                                                                                                   
                                                                                                                                              
    dataset_with_weather = dataset.copy()                                                                                                     
    for column in weather_keep_columns:                                                                                                       
        dataset_with_weather[column] = np.nan                                                                                                 
                                                                                                                                              
    for air_id in unique_air_store_ids:                                                                                                       
        station = air_nearest[air_nearest.air_store_id == air_id].station_id.iloc[0]                                                          
        weather_data = pd.read_csv(weather_data_dir + station + '.csv', parse_dates=['calendar_date']).rename(columns={'calendar_date': 'visit_date'})   
                                                                                                                                              
        this_store = dataset.store_id == air_id 
                                                                                                  
        merged = dataset[this_store].merge(weather_data, on='visit_date', how='left')                                                         
                                                                                                                                              
        for column in weather_keep_columns:                                                                                                   
            dataset_with_weather.loc[this_store, column] = merged[column]                                                                     
    return dataset_with_weather                                                                                                               
                                                                                                                                              
data = add_weather(data)                                                                                                                    
 

Adding weather...


ValueError: ignored

---

In [12]:
data_path = '/content/data/'
air_reserve = pd.read_csv(data_path + 'air_reserve.csv.zip').rename(columns={'air_store_id':'store_id'})
hpg_reserve = pd.read_csv(data_path + 'hpg_reserve.csv.zip').rename(columns={'hpg_store_id':'store_id'})
air_store = pd.read_csv(data_path + 'air_store_info.csv.zip').rename(columns={'air_store_id':'store_id'})
hpg_store = pd.read_csv(data_path + 'hpg_store_info.csv.zip').rename(columns={'hpg_store_id':'store_id'})
air_visit = pd.read_csv(data_path + 'air_visit_data.csv.zip').rename(columns={'air_store_id':'store_id'})
store_id_map = pd.read_csv(data_path + 'store_id_relation.csv.zip').set_index('hpg_store_id',drop=False)
date_info = pd.read_csv(data_path + 'date_info.csv.zip').rename(columns={'calendar_date': 'visit_date'}).drop('day_of_week',axis=1)
submission = pd.read_csv(data_path + 'sample_submission.csv.zip')

In [13]:
submission['visit_date'] = submission['id'].str[-10:]
submission['store_id'] = submission['id'].str[:-11]
air_reserve['visit_date'] = air_reserve['visit_datetime'].str[:10]
air_reserve['reserve_date'] = air_reserve['reserve_datetime'].str[:10]
air_reserve['dow'] = pd.to_datetime(air_reserve['visit_date']).dt.dayofweek
hpg_reserve['visit_date'] = hpg_reserve['visit_datetime'].str[:10]
hpg_reserve['reserve_date'] = hpg_reserve['reserve_datetime'].str[:10]
hpg_reserve['dow'] = pd.to_datetime(hpg_reserve['visit_date']).dt.dayofweek
air_visit['id'] = air_visit['store_id'] + '_' + air_visit['visit_date']
hpg_reserve['store_id'] = hpg_reserve['store_id'].map(store_id_map['air_store_id']).fillna(hpg_reserve['store_id'])
hpg_store['store_id'] = hpg_store['store_id'].map(store_id_map['air_store_id']).fillna(hpg_store['store_id'])
hpg_store.rename(columns={'hpg_genre_name':'air_genre_name','hpg_area_name':'air_area_name'},inplace=True)
data = pd.concat([air_visit, submission]).copy()
data['dow'] = pd.to_datetime(data['visit_date']).dt.dayofweek
date_info['holiday_flg2'] = pd.to_datetime(date_info['visit_date']).dt.dayofweek
date_info['holiday_flg2'] = ((date_info['holiday_flg2']>4) | (date_info['holiday_flg']==1)).astype(int)

air_store['air_area_name0'] = air_store['air_area_name'].apply(lambda x: x.split(' ')[0])

lbl = LabelEncoder()
air_store['air_genre_name'] = lbl.fit_transform(air_store['air_genre_name'])
air_store['air_area_name0'] = lbl.fit_transform(air_store['air_area_name0'])

data['visitors'] = np.log1p(data['visitors'])
data = data.merge(air_store,on='store_id',how='left')
data = data.merge(date_info[['visit_date','holiday_flg','holiday_flg2']], on=['visit_date'],how='left')

In [ ]:
air_store['air_area_name1'] = air_store['air_area_name'].apply(lambda x: ' '.join(x.split(' ')[:2]))
air_store['air_area_name1'] = lbl.fit_transform(air_store['air_area_name1'])
len(pd.unique(air_store['air_area_name1']))

55

In [ ]:
air_store[['store_id','air_area_name1']]
data = data.merge(air_store[['store_id','air_area_name1']],on='store_id',how='left')

In [ ]:
data['month'] = pd.to_datetime(data['visit_date']).dt.month

In [ ]:
data['expansion_after'] =((pd.to_datetime(data['visit_date']).dt.year >= 2016) | (pd.to_datetime(data['visit_date']).dt.month >= 7)).astype(int)


In [ ]:
data.tail()

,store_id,visit_date,visitors,id,dow,air_genre_name,air_area_name,latitude,longitude,air_area_name0,holiday_flg,holiday_flg2,air_area_name1,month,expansion_after
284122,air_fff68b929994bfbd,2017-05-27,0.0,air_fff68b929994bfbd_2017-05-27,5,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,1,39,5,1
284123,air_fff68b929994bfbd,2017-05-28,0.0,air_fff68b929994bfbd_2017-05-28,6,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,1,39,5,1
284124,air_fff68b929994bfbd,2017-05-29,0.0,air_fff68b929994bfbd_2017-05-29,0,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,0,39,5,1
284125,air_fff68b929994bfbd,2017-05-30,0.0,air_fff68b929994bfbd_2017-05-30,1,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,0,39,5,1
284126,air_fff68b929994bfbd,2017-05-31,0.0,air_fff68b929994bfbd_2017-05-31,2,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,0,39,5,1


#1. data 정제하기  

In [ ]:
train_size = len(air_visit)
test = data[train_size:]

restaurant = data.drop("latitude", axis=1).copy()
restaurant.drop("longitude", axis=1, inplace=True)
restaurant.drop("visit_date", axis=1, inplace=True)
restaurant.drop("id", axis=1, inplace=True)
restaurant.drop("store_id", axis=1, inplace=True)
restaurant.drop("air_area_name", axis=1, inplace=True)
restaurant.drop("air_area_name0", axis=1, inplace=True)


train_data = restaurant[:train_size]
test_data = restaurant[train_size:]



y_train = train_data["visitors"].copy()
X_train=train_data.drop("visitors", axis=1).copy() 

X_test = test_data.drop("visitors", axis=1)



In [ ]:
X_train

,dow,air_genre_name,holiday_flg,holiday_flg2,air_area_name1,month,expansion_after
0,2,4,0,0,37,1,1
1,3,4,0,0,37,1,1
2,4,4,0,0,37,1,1
3,5,4,0,1,37,1,1
4,0,4,0,0,37,1,1
...,...,...,...,...,...,...,...
252103,1,11,0,0,42,4,1
252104,2,11,0,0,42,4,1
252105,3,11,0,0,42,4,1
252106,4,11,0,0,42,4,1


# 2. Train 

## 2.1 Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor(learning_rate=0.2, random_state=3, n_estimators=200, subsample=0.8, max_depth =10)
GBR.fit(X_train, y_train)

predict=GBR.predict(X_test) # 0.7361

In [ ]:
print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [nan nan nan nan nan]
평균 검증 정확도: nan


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
KNR = KNeighborsRegressor(n_jobs=-1, n_neighbors=4)
KNR.fit(X_train, y_train)

predict=KNR.predict(X_test) # 0.82510

In [ ]:
from xgboost import XGBRegressor
XGB = XGBRegressor(learning_rate=0.2, random_state=3, n_estimators=280, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =12)
XGB.fit(X_train, y_train)
predict=XGB.predict(X_test) # 0.7368

[05:35:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
from sklearn.model_selection import cross_val_score, KFold
scores = cross_val_score(dt_clf , data , label , scoring='accuracy',cv=3)

In [ ]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train)
test
params = {
    
}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mse'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
gbm = lgb.train(params,lgb_train,2300)
predict = gbm.predict(X_test)  #0.78211



# 그리드 서치

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
LGBM = LGBMRegressor()
param_grid = {
    'learning rate': [1,0.1, 0.001, 0.003],
    'boosting_type': ['gbdt', 'rf'],
    'max_bin' : [10, 100, 100],
    'num_leaves': [10, 31, 127],
    
    
}
gbm = GridSearchCV(estimator=clf, param_grid=param_grid)
gbm.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'boosting_type': ['gbdt', 'rf'],
                         'learning rate': [0.1, 0.001, 0.003],
                         'max_bin': [10, 100, 100],
    

In [ ]:
print('best parameters : ', gbm.best_params_)
print('best score : ', gbm.best_score_)

best parameters :  {'boosting_type': 'gbdt', 'learning rate': 0.1, 'max_bin': 10, 'num_leaves': 10}
best score :  0.054162385346636824


In [ ]:
LGBM=gbm.best_estimator_
predict = gbm.predict(X_test)  #0.78389

# 3. submission

In [ ]:
predict

array([2.98269092, 2.7670509 , 2.84917391, ..., 2.04262467, 2.14783666,
       2.18055248])

In [ ]:
test['visitors'] = predict
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
test

,store_id,visit_date,visitors,id,dow,air_genre_name,air_area_name,latitude,longitude,air_area_name0,holiday_flg,holiday_flg2,air_area_name1,month,expansion_after
252108,air_00a91d42b08b08d9,2017-04-23,18.740866,air_00a91d42b08b08d9_2017-04-23,6,6,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,7,0,1,26,4,1
252109,air_00a91d42b08b08d9,2017-04-24,14.911640,air_00a91d42b08b08d9_2017-04-24,0,6,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,7,0,0,26,4,1
252110,air_00a91d42b08b08d9,2017-04-25,16.273506,air_00a91d42b08b08d9_2017-04-25,1,6,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,7,0,0,26,4,1
252111,air_00a91d42b08b08d9,2017-04-26,18.072686,air_00a91d42b08b08d9_2017-04-26,2,6,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,7,0,0,26,4,1
252112,air_00a91d42b08b08d9,2017-04-27,18.039348,air_00a91d42b08b08d9_2017-04-27,3,6,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,7,0,0,26,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284122,air_fff68b929994bfbd,2017-05-27,12.096508,air_fff68b929994bfbd_2017-05-27,5,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,1,39,5,1
284123,air_fff68b929994bfbd,2017-05-28,10.328028,air_fff68b929994bfbd_2017-05-28,6,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,1,39,5,1
284124,air_fff68b929994bfbd,2017-05-29,6.710821,air_fff68b929994bfbd_2017-05-29,0,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,0,39,5,1
284125,air_fff68b929994bfbd,2017-05-30,7.566306,air_fff68b929994bfbd_2017-05-30,1,1,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,7,0,0,39,5,1


In [ ]:
sub = test[['id','visitors']].copy()
sub.to_csv('submission.csv', index=False)

In [ ]:
import lightgbm as lgb
def lightgbm(data):
    train_size = len(air_visit)
    test = data[train_size:]

    restaurant = data.drop("latitude", axis=1).copy()
    restaurant.drop("longitude", axis=1, inplace=True)
    restaurant.drop("visit_date", axis=1, inplace=True)
    restaurant.drop("id", axis=1, inplace=True)
    restaurant.drop("store_id", axis=1, inplace=True)
    restaurant.drop("air_area_name", axis=1, inplace=True)
    restaurant.drop("air_area_name0", axis=1, inplace=True)

    train_data = restaurant[:train_size]
    test_data = restaurant[train_size:]

    y_train = train_data["visitors"].copy()
    X_train=train_data.drop("visitors", axis=1).copy() 

    X_test = test_data.drop("visitors", axis=1)
    
    lgb_train = lgb.Dataset(X_train, y_train)

    params = {}
    params['learning_rate'] = 0.003
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'regression'
    params['metric'] = 'mse'
    params['sub_feature'] = 0.5
    params['num_leaves'] = 10
    params['min_data'] = 50
    params['max_depth'] = 10
    gbm = lgb.train(params,lgb_train,2300)
    predict = gbm.predict(X_test)  #0.78211
    test['visitors'] = predict
    test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
    sub = test[['id','visitors']].copy()
    sub.to_csv('submission.csv', index=False)



In [ ]:
lightgbm(data)